In [ ]:
!pip install morfessor
!pip install polyglot
!pip install pdfminer.six
!pip install pycld2
!pip install pyicu

In [6]:
import pandas as pd
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

df = pd.read_csv("datasample.csv")
le = preprocessing.LabelEncoder()
df['category'] = le.fit_transform(df["category"])
df['type'] = le.fit_transform(df['type'])
df_new = df[['unique_num', 'type']]
x_train, x_test, y_train, y_test = train_test_split(df_new, df['category'], test_size=0.3, random_state=109)
model = GaussianNB()
model.fit(x_train, y_train)
report = classification_report(y_test, model.predict(x_test), target_names=['Биометреческие', 'Общедоступные', 'Специальные'])
print(report)

                precision    recall  f1-score   support

Биометреческие       0.75      1.00      0.86        59
 Общедоступные       1.00      1.00      1.00       126
   Специальные       1.00      0.83      0.90       115

      accuracy                           0.93       300
     macro avg       0.92      0.94      0.92       300
  weighted avg       0.95      0.93      0.93       300



In [ ]:
import pdfminer.high_level
import polyglot
from polyglot.text import Text, Word
from polyglot.detect.base import logger as polyglot_logger
import re

polyglot_logger.setLevel("ERROR")

extract_text = ""
with open('example.pdf', 'rb') as file:
    extract_text = pdfminer.high_level.extract_text(file)
polyglot_text = Text(extract_text)
text_lowcase_no_marks = []
for token in polyglot_text.words:
    if (token.isalpha()):
        text_lowcase_no_marks.append(token.lower())
#print(text_lowcase_no_marks)
personal_data = []
other_pdn = []
for word in text_lowcase_no_marks:
    if re.match(r'^фамили.', word) and not (0, "фамилия") in personal_data:
        personal_data.append((0, "фамилия"))
    elif re.match(r'^им.*', word) and not (1, "имя") in personal_data:
        personal_data.append((1, "имя"))
    elif re.match(r'^отчеств.', word) and not (2, "отчество") in personal_data:
        personal_data.append((2, "отчество"))
    elif re.match(r'^рожден.*', word) and not (3, "дата и место рождения") in personal_data:
        personal_data.append((3, "дата и место рождения"))
    elif re.match(r'^житель.*', word) and not (4, "адрес проживания") in personal_data:
        personal_data.append((4, "адрес проживания"))
    elif re.match(r'^професси.$', word) and not (5, "профессия") in personal_data:
        personal_data.append((5, "профессия"))
    elif re.match(r'^работ.$', word) and not (6, "место работы") in personal_data:
        personal_data.append((6, "место работы"))
    elif re.match(r'^электронн.*$', word) and not (7, "адрес электронной почты") in personal_data:
        personal_data.append((7, "адрес электронной почты"))
    elif re.match(r'^телефон.*$', word) and not (8, "телефон") in personal_data:
        personal_data.append((8, "телефон"))
    elif re.match(r'^образован.*$', word) or re.match(r'^диплом.*$', word) and not (9, "образование") in personal_data:
        personal_data.append((9, "образование"))
    elif re.match(r'^секс.*$', word) and not (10, "сексуальная жизнь") in personal_data:
        personal_data.append((10, "сексуальная жизнь"))
    elif re.match(r'^полити.*$', word) and not (11, "политические взгляды") in personal_data:
        personal_data.append((11, "политические взгляды"))
    elif re.match(r'^религи.*$', word) or re.match(r'^конфесси.*$', word) and not (12, "философские/религиозные взгляды") in personal_data:
        personal_data.append((12, "философские/религиозные взгляды"))
    elif re.match(r'^рас.$', word) or re.match(r'^расов..$', word) and not (13, "раса") in personal_data:
        personal_data.append((13, "раса"))
    elif re.match(r'^пол.?$', word) or re.match(r'^полов..$', word) and not (14, "пол") in personal_data:
        personal_data.append((14, "пол"))
    elif re.match(r'^национальност.*$', word) and not (15, "национальность") in personal_data:
        personal_data.append((15, "национальность"))
    elif re.match(r'^днк$', word) and not (16, "днк") in personal_data:
        personal_data.append((16, "днк"))
    elif re.match(r'^фотограф.*$', word) and not (17, "фотография лица") in personal_data:
        personal_data.append((17, "фотография лица"))
    elif re.match(r'^кров.*$', word) and not (18, "группа крови") in personal_data:
        personal_data.append((18, "группа крови"))
    elif re.match(r'^глаз.*$', word) and not (19, "сетчатка глаза") in personal_data:
        personal_data.append((19, "сетчатка глаза"))
    elif re.match(r'^пальц.*$', word) and not (20, "отпечатки пальцев") in personal_data:
        personal_data.append((20, "отпечатки пальцев"))
    elif re.match(r'^платеж.*$', word) and not ("платежная информация", "иные") in other_pdn:
        other_pdn.append(("платежная информация", "иные"))
    elif re.match(r'^зарпла.*$', word) and not ("зарплата", "иные") in other_pdn:
        other_pdn.append(("зарплата", "иные"))
    else:
        continue
#personal_data_transform = le.fit_transform(personal_data)
pdn1 = [x[0] for x in personal_data]
pdn2 = [x[1] for x in personal_data]
data1 =  {'unique_num' : pdn1,
         'type': pdn2}
personal_data_no_transform = pd.DataFrame(data1)
pdn2_transform = le.fit_transform(pdn2)
data2 =  {'unique_num' : pdn1,
         'type': pdn2_transform}
end_df = pd.DataFrame(data2)
classification = model.predict(end_df)
personal_data_no_transform['category'] = classification
classes_transform = []
for index in range(len(personal_data_no_transform['category'])):
    if (classification[index] == 1):
        classes_transform.append("общедоступные")
    elif (classification[index] == 2):
        classes_transform.append("специальные")
    elif (classification[index] == 0):
        classes_transform.append("биометрические")
personal_data_no_transform['category'] = classes_transform
personal_data_no_transform.drop('unique_num', inplace=True, axis = 1)
for element in other_pdn:
    new_row = {'type': element[0], 'category': element[1]}
    personal_data_no_transform = personal_data_no_transform.append(new_row, ignore_index = True)
personal_data_no_transform

,type,category
0,фамилия,общедоступные
1,имя,общедоступные
2,отчество,общедоступные
3,адрес проживания,общедоступные
4,дата и место рождения,общедоступные
5,телефон,общедоступные
6,платежная информация,иные
